# Hand digit classifier 
---
## Incremental network quantization

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from skimage import io

In [ ]:
def getBounderyExponents(W, b):
    s = torch.max(torch.abs(W)).item()
    n1 = np.floor(np.log2(4*(s/3)))
    n2 = n1 + 1 - (2**(b - 1))/2
    return n1, n2

def getQuantizationMask(W, percentage, T):
    w = W.view(-1)
    t = T.view(-1)
    idx = t == 1

    numberOfWeights = w.size(dim=0)
    numberOfQWeights = int(percentage*numberOfWeights - t[idx].size(dim=0))

    t_aux = torch.Tensor(np.ones_like(T)).view(-1)
    w = w*(t_aux - t)
    w = torch.abs(w)
    sorted_w, indices_w = w.sort()
    t[indices_w[-numberOfQWeights:]] = 1
    
    return t.view(T.size())

def quantizeWeights(W, T, n1, n2):
    T_aux = torch.Tensor(np.ones_like(T))
    eps = 1e-6
    W1 = W*(T_aux - T)
    idx = W == 0
    W[idx] = eps

    closestExp = np.floor(np.log2(np.abs(W*4/3)))
    Q = W1 + (2**closestExp)*T

    idx = closestExp < n2
    Q[idx] = 0
    idx = closestExp > n1
    Q[idx] = 2**n1

    return closestExp, Q

In [112]:
W = torch.tensor(np.array([
    [0.01, 0.02, -0.2, 0.04, 0.33],
    [0.17, -0.42, -0.33, 0.02, -0.05], 
    [0.02, 0.83, -0.03, 0.03, 0.06],
    [-0.9, 0.07, 0.11, 0.87, -0.36], 
    [-0.73, 0.41, 0.42, 0.39, 0.47]]))
bit_length = 4
n1, n2 = getBounderyExponents(W, bit_length)
print(n1, n2)
T = torch.Tensor(np.zeros_like(W))
T = getQuantizationMask(W, 0.5, T)

T_aux = torch.Tensor(np.ones_like(T))
eps = 1e-6
W1 = W*(T_aux - T)
W2 = W*T + eps*(T_aux - T)
#print(W2)
closestExp = np.floor(np.log2(np.abs(W2*4/3)))
print(W1 + (2**closestExp)*T)
print(closestExp)
Q = W1 + (2**closestExp)*T
idx = closestExp < -1
Q[idx] = 0
Q

0.0 -3.0
tensor([[ 0.0100,  0.0200, -0.2000,  0.0400,  0.2500],
        [ 0.1700,  0.5000,  0.2500,  0.0200, -0.0500],
        [ 0.0200,  1.0000, -0.0300,  0.0300,  0.0600],
        [ 1.0000,  0.0700,  0.1100,  1.0000,  0.2500],
        [ 0.5000,  0.5000,  0.5000,  0.5000,  0.5000]], dtype=torch.float64)
tensor([[-20., -20., -20., -20.,  -2.],
        [-20.,  -1.,  -2., -20., -20.],
        [-20.,   0., -20., -20., -20.],
        [  0., -20., -20.,   0.,  -2.],
        [ -1.,  -1.,  -1.,  -1.,  -1.]], dtype=torch.float64)


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 1.0000, 0.0000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000]], dtype=torch.float64)

In [ ]:
weight = np.array([-0.02, 0.87])

closestExp = np.floor(np.log2(np.abs(weight*4/3)))
print("{} -> {}*2^{}".format(weight, np.sign(weight), closestExp))